In [359]:
import time
import xlrd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import excelSave as save
import datetime
from datetime import datetime
from datetime import timedelta
import dateutil.relativedelta
import calendar
import re

我手头的项目是要爬2010年到现在每个月/每一天的关于同居的微博。目前常见的微博爬虫有三个目标网站，一个是pc端口s.weibo.com，一个是智能手机端口m.weibo.cn，一个是非常老的手机端口weibo.cn。很多微博爬虫都采用智能手机端端口，因为可以爬到很多数据，只要不停往下滑应该就能爬完。但是我目前还没有发现智能手机端口如何搜索指定的日期或时间段，这就不得不用pc端口或者非常老的那个手机端口了。

非常老的手机端口的好处在于不用管‘展开全文’，pc端需要展开原文。而且每一个日期可以搜索到100页，pc端只能搜索到50页。虽然可能pc端每一页比手机端口要多，但是应该还是这个端口能爬到的更多一点。第三个原因是可以直接勾选原创微博，省去了看上去爬了很多，但实际上爬完了还需要删除所有的转发的过程（虽然复制粘贴的还是要额外删除）。

用selenium是因为微博会反爬虫，selenium可以模拟正常登陆的流程，而且一定要在每一次点击一个页面之后停顿几秒钟（sleep），不然很容易被封号甚至封IP。

In [170]:
#这是每个月的开始结束日期，我先试试每个月的，这样样本数量小一点
#This is monthly start/enddate, it has much smaller sample.I'll do it for now

YEAR = 2010
MONTH = 1
DAY = 1

date = datetime(YEAR,MONTH,DAY)
startdate = []
for i in range(12):
    year = date.strftime("%Y")
    month = date.strftime("%m")
    day = date.strftime("%d")
    #变成string turn date into string
    startstring = year+month+day
    startdate.append(startstring)
    
    #下一个月 Move forward by month
    date = datetime(date.year,date.month,date.day) + dateutil.relativedelta.relativedelta(months=1)

date = datetime(YEAR,MONTH,DAY)
enddate = []
for i in range(12):
    year = date.strftime("%Y")
    month = date.strftime("%m")
    #用calender取月末  Last day in month using calender
    day = str(calendar.monthrange(date.year,date.month)[1])
    #变成string turn date into string
    startstring = year+month+day
    enddate.append(startstring)
    
    #下一个月 Move forward by month
    date = datetime(date.year,date.month,date.day) + dateutil.relativedelta.relativedelta(months=1)
    
date = datetime(YEAR,MONTH,DAY)

In [171]:
enddate

['20100131',
 '20100228',
 '20100331',
 '20100430',
 '20100531',
 '20100630',
 '20100731',
 '20100831',
 '20100930',
 '20101031',
 '20101130',
 '20101231']

#我打算最终的项目一年一年的下载数据 I plan on scraping yealy data for my final project but not yet
#这是一年里的每天 This is everyday in a year
#每天的日期储存在days里 For reiteration through each day
#还没写完 unfinished

date = datetime(2010,1,1)
days = []

for i in range(35): 
    year = date.strftime("%Y")
    month = date.strftime("%m")
    day = date.strftime("%d")
    datestring = year+month+day
    days.append(datestring)
    
    #加一天 add one day
    date += timedelta(days=1)
    
date = datetime(2010,1,1)

In [360]:
if __name__ == '__main__':
    username = "17064888409" #用户名username
    password = "rw297421" #密码password
    driver = webdriver.Chrome()#可以换成firefox.Can use firefox
    
    
#登陆 Login
driver.get('https://weibo.cn/pub/?vt=' )
   
elem = driver.find_element_by_xpath("//*[@class='ut']/a[1]").click();
time.sleep(1)
    
print("开始自动登陆，若出现验证码手动验证.Loading")
time.sleep(1)

elem = driver.find_element_by_xpath("//*[@id='loginName']");
elem.send_keys(username)
elem = driver.find_element_by_xpath("//*[@id='loginPassword']");
elem.send_keys(password)
elem = driver.find_element_by_xpath("//*[@id='loginAction']");
elem.send_keys(Keys.ENTER)  
print("暂停5秒，用于验证码验证")
time.sleep(5)

开始自动登陆，若出现验证码手动验证.Loading
暂停5秒，用于验证码验证


In [378]:
#点开搜索页面 Open search tap
driver.find_element_by_xpath("//a[contains(text(),'搜索')]").click()
driver.find_element_by_xpath("//a[contains(text(),'高级搜索')]").click()
time.sleep(1)

#输入关键词 Input Keywords
#我这里是"同居" In this case 'cohabitation'
elem = driver.find_element_by_xpath("//*[@type='text']")
elem.send_keys('同居')

#勾选原创 Click original posts
driver.find_element_by_xpath("//*[@name='hasori']").click()

#开始年月日 Start data e.g, 20200101
elem = driver.find_element_by_xpath("//*[@name='starttime']")
elem.send_keys(startdate)

#结束年月日 End data e.g, 20200131
elem = driver.find_element_by_xpath("//*[@name='endtime']")
elem.clear()
elem.send_keys(enddate)
time.sleep(1)

#搜索 Search
driver.find_element_by_xpath("//*[@type='submit']").click()

In [279]:
#开始爬微博scraping,注意elements是可以抓取多个，element只能抓一个
#start scraping. Notice elements not element. Elements can find multiple element

usernames = []
user_links = []
posts = []
likes = []
reblogs = []
comments = []
comment_links = []
post_date = []
post_time = []

def scrape_one_page(driver):
    #找到发微博的用户名和连接
    #Get username and href link
    elem = driver.find_elements_by_xpath("//*[@class='nk']")
    for post in elem:
        usernames.append(post.text)
        user_links.append(post.get_attribute('href'))

    #找到微博内容
    #Get post content
    elem = driver.find_elements_by_xpath("//*[@class='ctt']")
    for post in elem:
        posts.append(post.text)

    #点赞
    #likes
    likes_temp = []
    elem = driver.find_elements_by_xpath("//a[contains(text(),'赞')]")
    for like in elem:
        likes_temp.append(like.text)
    for like in likes_temp:
        like = re.sub(r'\D', '', like) #去除掉‘赞’这个字只保留数字Delete 'likes' text
        likes.append(like)
    
    #转发评论
    #reblogs
    reblog_temp = []
    elem = driver.find_elements_by_xpath("//a[contains(text(),'转发')]")
    for reblog in elem:
        reblog_temp.append(reblog.text)
    for reblog in reblog_temp:
        reblog = re.sub(r'\D', '', reblog) #保留数字keep the number
        reblogs.append(reblog)  

    #评论和评论的链接comment and links to comment section
    comment_temp = []
    elem = driver.find_elements_by_xpath("//a[contains(text(),'评论')]")
    for comment in elem:
        comment_temp.append(comment.text)
        comment_links.append(comment.get_attribute('href'))#评论的链接links to comment
    for comment in comment_temp:
        comment = re.sub(r'\D', '', comment) #保留数字keep the number
        comments.append(comment)  

    #发布日期和时间
    post_date = []
    elem = driver.find_elements_by_xpath("//*[@class='ct']")
    for time in elem:
        post_date.append(time.text.split(' ')[0])
        post_time.append(time.text.split(' ')[1])


In [280]:
scrape_one_page(driver)

In [255]:
#一页爬完了爬下一页Next page
#driver.find_element_by_xpath("//a[contains(text(),'下页')]").click()
NextPage = driver.find_element_by_xpath("//*[@class='pa']/form/div/a[1]")
NextPage.click()
time.sleep(0.5)

In [353]:
elem = driver.find_elements_by_xpath("//*[@class='pa']")
for page in elem:
    page = page.text.split(' ')[4:]
    for p in page:
        page_number = p[:-1]
        page_number = page_number.split('/')
        current_page = int(page_number[0])
        max_page = int(page_number[1])

In [355]:
if current_page == max_page:
   #点开‘更多’ Open'more'
    driver.find_element_by_xpath("//a[contains(text(),'更多')]").click()
    
    #开始年月日 Start data e.g, 20200101
    elem = driver.find_element_by_xpath("//*[@name='starttime']")
    elem.send_keys(startdate)

    #结束年月日 End data e.g, 20200131
    elem = driver.find_element_by_xpath("//*[@name='endtime']")
    elem.clear()
    elem.send_keys(enddate)
    time.sleep(1)

    #搜索 Search
    driver.find_element_by_xpath("//*[@type='submit']").click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@name='hasori']"}
  (Session info: chrome=80.0.3987.122)


In [341]:
x = '4/65页'

In [342]:
x= x.split('/')
x[0]

'4'

In [333]:
x[:-1]-1

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [322]:
a = 65/65-1 

In [328]:
if a == 0:
    b = 1
b


1

In [266]:
elem = driver.find_elements_by_xpath("//*[@class='pa']")
page_number = []
for post in elem:
    page_number.append(post.text)
page_number

['下页 上页 首页  4/65页']

In [271]:
for a in page_number:
    a = a.split(' ')
    page_number.append(a)
page_number

AttributeError: 'list' object has no attribute 'split'

In [241]:
pattern = re.compile(r'\d+\/\d+页')

elements = driver.find_elements_by_css_selector("type.submit")
for element in elements:
    match = pattern.match(element.text)
    if match:
        print(element.text)

In [157]:
print(likes)
print(reblogs)
print(comments)

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
['0', '0', '0', '0', '4', '0', '0', '0', '0', '0']
['1', '0', '0', '5', '10', '0', '0', '0', '0', '0']


In [147]:
post_date = []
elem = driver.find_elements_by_xpath("//*[@class='ct']")
for time in elem:
    post_date.append(time.text.split(' ')[0])
post_date 

['2010-01-31',
 '2010-01-31',
 '2010-01-31',
 '2010-01-31',
 '2010-01-31',
 '2010-01-31',
 '2010-01-31',
 '2010-01-31',
 '2010-01-31',
 '2010-01-31']

In [275]:
post_date

[]

In [33]:
usernames = []
userlink = []
username = driver.find_elements_by_xpath("//*[@class='nk']")
for a in username:
    usernames.append(a.text)
    userlink.append(a.get_attribute('href'))

In [53]:
likes = []
elem = driver.find_elements_by_xpath("//*[@class='c']/div[1]/a[2]")
for like in elem:
    like = re.findall(r'\d+', like.text) 
    likes.append(like)
likes

[['0'], ['0'], ['0'], ['0'], ['0'], ['0'], ['0'], ['0']]

In [66]:
likes = []
likes_temp = []
elem = driver.find_elements_by_xpath("//*[@class='c']/div[1]/a[2]")
for like in elem:
    likes_temp.append(like.text)
for like in likes_temp:
    like = re.sub(r'\D', '', like)
    likes.append(like)
likes

['0', '0', '0', '0', '0', '0', '0', '0']

In [81]:
a = '评论[1]'
a = re.sub(r'\D', '',a)
a

'1'

In [58]:
for like in likes:
    like.replace(r'(?!\d)','' , like)
likes

TypeError: 'str' object cannot be interpreted as an integer

In [ ]:
likes = []
elem = driver.find_elements_by_xpath("//*[@class='c']/div[1]/a[2]")
for like in elem:
    like = re.findall(r'\d+', like.text) 
    likes.append(like)
likes

In [27]:
print(posts)

[':婚礼不是同居的广告。鲁迅', ':昨日，一河南商丘22岁男子身体突然起火并严重烧伤其同居女友，目前2人在医院观察，男子口不能言。在世界上，人体自燃现象十分罕见，但也事实存在，有很多人都是自身突然起火然后烧个精光，侥幸存活的很少。在大陆这是首次。', ':2010年1月16日，位于郑州的中原工学院内的一间教室内，一场名为《你我健康青春同伴教育》的青年志愿者活动正在进行。志愿者通过“大学生同居能不能被理解？”的讨论，与大学生讨论正确的恋爱观问题。', ':张敬轩其实一直很喜欢，从第一次登台唱(过云雨)，到后来迷上他的打扮。再到后来知道他和关智斌同居。原来我的雷达没有错，果然是同路人', ':原来列侬也有小三，她的名字叫May Pang,自1973年夏至1975年初，与Lennon相爱并同居在一起。Pang是美国第二代移民，父母来自江浙，并为她起了一个中国名字：庞凤仪。', ':@xuechengfashi 优婆夷说：凡是来这里见我色相的，听我音声的，记念我的，或者与我同居共住的，给我服事的，都不会空过。他们一切的病苦除灭了，烦恼远离了，进入无碍的清净。我增长他们的善根，引他们走入功德之门。', ':#房子#家也不算小，却要与肉同居，到处都是东西，箱子，柜子，人给物让位，颠倒颠倒，还越来越多，外来物种入侵。', ':六大成绩:房产改革口袋掏空 教育改革父母逼疯 医疗改革提前送终 企业改革下岗停工 政府改革机构臃肿 婚姻改革同居成风', ':我的女朋友也叫丫头。。。 好书推荐：《白领手记：和空姐同居的日子》 http://sinaurl.cn/hIkyO', ':GLEE+LU+NYILU+Bro+TVD+好想+家教+毀滅六人+近所+人龍+新同居 以上先']


In [6]:
#创建文件
    if os.path.exists(book_name_xls):
        print("文件已存在")
    else:
        print("文件不存在，重新创建")
        value_title = [["rid", "用户名称Username", "微博等级", 
                        "微博内容", "微博转发量","微博评论量",
                        "微博点赞","发布时间","搜索关键词",
                        "话题名称","话题讨论数","话题阅读数"],]
        save.write_excel_xls(book_name_xls, sheet_name_xls, 
                             value_title)

IndentationError: unexpected indent (<ipython-input-6-c6754c682654>, line 2)

In [ ]:
    
    yuedu_taolun = driver.find_element_by_xpath("//*[@id='app']/div[1]/div[1]/div[2]/div/div/div[2]/h4[2]").text
    yuedu = yuedu_taolun.split(" ")[0]
    taolun = yuedu_taolun.split(" ")[1]
    time.sleep(2)
    name = keyword
    shishi_element = driver.find_element_by_xpath("//*[@class='scroll-box nav_item']/ul/li/span[text()='实时']")
    driver.execute_script('arguments[0].click()',shishi_element) 
    get_current_weibo_data(elems,book_name_xls,name,yuedu,taolun,maxWeibo) #爬取实时
    time.sleep(2)
    

In [ ]:
def spider(username,password,driver,book_name_xls,sheet_name_xls,keyword,maxWeibo):
    
    #创建文件
    if os.path.exists(book_name_xls):
        print("文件已存在")
    else:
        print("文件不存在，重新创建")
        value_title = [["rid", "用户名称", "微博等级", "微博内容", "微博转发量","微博评论量","微博点赞","发布时间","搜索关键词","话题名称","话题讨论数","话题阅读数"],]
        save.write_excel_xls(book_name_xls, sheet_name_xls, value_title)
    
    #加载驱动，使用浏览器打开指定网址  
    driver.set_window_size(452, 790)
    driver.get('https://weibo.cn/pub/?vt=' )
   
    elem = driver.find_element_by_xpath("//*[@class='ut']/div[1]").click();
    time.sleep(1)
    
    print("开始自动登陆，若出现验证码手动验证")
    time.sleep(1)
    
    
    #登陆
    elem = driver.find_element_by_xpath("//*[@id='loginName']");
    elem.send_keys(username)
    elem = driver.find_element_by_xpath("//*[@id='loginPassword']");
    elem.send_keys(password)
    elem = driver.find_element_by_xpath("//*[@id='loginAction']");
    elem.send_keys(Keys.ENTER)  
    print("暂停20秒，用于验证码验证")
    time.sleep(20)

    
    #判断页面是否加载出
    while 1:  # 循环条件为1必定成立
        result = isPresent()
        print ('判断页面1成功 0失败  结果是=%d' % result )
        if result == 1:
            elems = driver.find_elements_by_css_selector('div.line-around.layout-box.mod-pagination > a:nth-child(2) > div > select > option')
            #return elems #如果封装函数，返回页面
            break
        else:
            print ('页面还没加载出来呢')
            time.sleep(20)
    time.sleep(2)
    
    #搜索关键词
    elem = driver.find_element_by_xpath("//*[@class='m-text-cut']").click();
    time.sleep(2)
    elem = driver.find_element_by_xpath("//*[@type='search']");
    elem.send_keys(keyword)
    elem.send_keys(Keys.ENTER)

    time.sleep(5)
    #elem = driver.find_element_by_xpath("//*[@id='app']/div[1]/div[1]/div[3]/div/div/div/div[1]/span[2]/span[1]")

    #elem.click()
    print("超话链接获取完毕，休眠2秒")
    time.sleep(2)
    
    yuedu_taolun = driver.find_element_by_xpath("//*[@id='app']/div[1]/div[1]/div[2]/div/div/div[2]/h4[2]").text
    yuedu = yuedu_taolun.split(" ")[0]
    taolun = yuedu_taolun.split(" ")[1]
    time.sleep(2)
    name = keyword
    shishi_element = driver.find_element_by_xpath("//*[@class='scroll-box nav_item']/ul/li/span[text()='实时']")
    driver.execute_script('arguments[0].click()',shishi_element) 
    get_current_weibo_data(elems,book_name_xls,name,yuedu,taolun,maxWeibo) #爬取实时
    time.sleep(2)
    
if __name__ == '__main__':
    username = "17064888409" #你的微博登录名
    password = "rw297421" #你的密码
    driver = webdriver.Chrome()#你的chromedriver的地址
    book_name_xls = "#人大法工委回应未婚同居合法化#.xls" #填写你想存放excel的路径，没有文件会自动创建
    sheet_name_xls = '微博数据' #sheet表名
    maxWeibo = 5000 #设置最多多少条微博
    keywords = ["#人大法工委回应未婚同居合法化#"] 
    for keyword in keywords:
        spider(username,password,driver,book_name_xls,sheet_name_xls,keyword,maxWeibo)

In [ ]:
elem = driver.find_element_by_xpath("//*[@class='ut']/a[1]").click();
time.sleep(1)

In [ ]:
#点开搜索页面 Open search tap
driver.find_element_by_xpath("//*[@class='n']/a[4]").click()
time.sleep(1)

#输入关键词 Input Keywords
#我这里是"同居" In this case 'cohabitation'
elem = driver.find_element_by_xpath("//*[@type='text']")
elem.send_keys('同居')
elem.send_keys(Keys.ENTER)